In [1]:
from  pathlib import Path
from helper import load_dotenv
import re
import shutil
from tqdm import tqdm

In [2]:
config = load_dotenv()

# PDFs verschieben

In [ ]:
pdf_dir = Path('/Volumes/ctp_00/_temp/frederic_moire_tests/')

In [ ]:
pdf_dir.exists()

In [ ]:
pdfs = list( pdf_dir.glob('./*.pdf') )

In [ ]:
len(pdfs)

In [ ]:
for pdf in tqdm(pdfs):
    job_name = pdf.name[:6]

    target_dir = config['DATA_DIR'] / job_name
    target_path = target_dir / pdf.name
    
    if target_dir.exists() == False:
        target_dir.mkdir()
    
    shutil.move(
        pdf,
        target_path
    )

# Dateien sortieren

In [3]:
job_dirs = [d for d in config['DATA_DIR'].iterdir() if d.is_dir() and re.match('\d{6}.*', d.name)]

In [17]:
def process_job_dir( j_dir ):
    for pdf in list(j_dir.glob('./*.pdf')):
        res = re.match( r'(.+)__(\d+)__#__(.+)', pdf.name )
    
        if res:
            job, lpi, filename = res.groups()
            basename = filename.strip( pdf.suffix )
    
            relevant_files = list( j_dir.glob(f'./{ basename }*') )
    
            # dirs anlegen
            pdf_dir = j_dir / 'pdf'
            img_dir = j_dir / f'vps2400dpi{ lpi }lpi'
    
            if pdf_dir.exists() == False:
                pdf_dir.mkdir()
    
            if img_dir.exists() == False:
                img_dir.mkdir()
    
            # dateien verschieben
            pdf_out_file = pdf_dir / filename
            shutil.move( pdf, pdf_out_file )
    
            for rf in relevant_files:
                out_file = img_dir / rf.name
    
                shutil.move(rf, out_file)

In [18]:
for j_dir in tqdm( job_dirs ):
    process_job_dir( j_dir )

100%|███████████████████████████████████████████| 41/41 [00:05<00:00,  7.78it/s]


In [21]:
from add_data_to_db import main as import_data
from add_jpgs_to_labelstudio import main as add_jpgs_to_labelstudio

In [22]:
import_data()
add_jpgs_to_labelstudio()

100%|███████████████████████████████████████| 2667/2667 [00:32<00:00, 82.57it/s]
